# 6.3 - Modeling Correlated Supply Disruptions
Prepared by: Nickolas Freeman, PhD

This notebook demonstrates how to incorporate correlated supplier disruptions when limited data is available. In such cases, the fact that limited data is available may lead decision-makers to doubt the correlation estimates suggested by the data. The following code block imports some libraries that we will use.

In [ ]:
import pathlib

from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pulp
import seaborn as sns
sns.set_style('whitegrid')

import OM527_functions as omf

Before looking at the sourcing models, we will consider how data can sometimes not provide a complete picture of the relationships that exist among variables. The following code block reads a files that contains 13 datasets. The first and last five rows are printed.

In [ ]:
dozen_datapath = pathlib.Path('data', 'Dozen.csv')
dozen = pd.read_csv(dozen_datapath)
dataset_names = dozen['dataset'].unique().tolist()
dozen

The following code block prints the mean, meadian, standard deviation, and corrleation for the x and y values associated with the datasets.

In [ ]:
dataset_summary = {}
rounding_digits = 3
for current_dataset_name in dataset_names:
    current_dataset = dozen[dozen['dataset'] == current_dataset_name]
    dataset_summary[current_dataset_name] = {
        'x_mean': current_dataset['x'].mean(), 
        'y_mean': current_dataset['y'].mean(), 
        'x_median': current_dataset['x'].median(), 
        'y_median': current_dataset['y'].median(), 
        'x_std': current_dataset['x'].std(axis = 0), 
        'y_std': current_dataset['y'].std(axis = 0), 
        'correlation': current_dataset[['x', 'y']].corr().values[0][1],
    }
    
pd.DataFrame().from_dict(dataset_summary, orient = 'index').round(rounding_digits)    

The following code block generates and interactive visualization that allows us to view the datasets.

In [ ]:
dataset_names = dozen['dataset'].unique().tolist()

@interact(current_dataset_name = dataset_names)

def interactive_spend_analysis(current_dataset_name = dataset_names[0]):

    current_dataset = dozen[dozen['dataset'] == current_dataset_name]

    fig, ax = plt.subplots(1, 1, figsize = (8,6))
    sns.scatterplot(x = 'x', 
                    y = 'y', 
                    data = current_dataset)
    plt.show()

We will consider cases where the on-time dlivery rate for suppliers are correlated. The following code block defines an optimization model that maximizes the buyer's expected profit under the assumption that demand is uncertain.

In [ ]:
def solve_OTD_model(cost_vector, otd_sample, demand_sample, sales_price):
    
    num_scenarios = len(demand_sample)
    num_suppliers = len(cost_vector)
    
    prob = pulp.LpProblem("Newsvendor_Problem", pulp.LpMaximize)

    Order = {} 
    for supplier in range(num_suppliers):
        Order[supplier] = pulp.LpVariable(f"Order_from_{supplier}", 
                                          lowBound = 0, 
                                          upBound = None, 
                                          cat = pulp.LpContinuous)

    Sell = {}
    for scenario in range(num_scenarios):
        Sell[scenario] = pulp.LpVariable(f"Sell_in_Scenario_{scenario}", 
                                         lowBound = 0, 
                                         upBound = None, 
                                         cat = pulp.LpContinuous)

    expected_revenue = []
    for scenario in range(num_scenarios):
        expected_revenue.append(sales_price*Sell[scenario])
    expected_revenue = pulp.lpSum(expected_revenue)/num_scenarios

    expected_cost = []
    for scenario in range(num_scenarios): 
        for supplier in range(num_suppliers):
            expected_cost.append(cost_vector[supplier]\
                                 *Order[supplier]\
                                 *otd_sample[scenario, supplier])
    expected_cost = pulp.lpSum(expected_cost)/num_scenarios

    objective_function = expected_revenue - expected_cost
    prob += objective_function

    for scenario in range(num_scenarios):
        total_delivered = pulp.lpSum([Order[supplier]*otd_sample[scenario, supplier]
                                      for supplier in range(num_suppliers)]) 
        prob += (Sell[scenario] <= total_delivered, f'Sell{scenario} <= Order')

    for scenario in range(num_scenarios):
        prob += (Sell[scenario] <= demand_sample[scenario], f'Sell{scenario} <= Demand{scenario}')
        
    prob.solve()    
       
    solution_dict = {}
    for supplier in range(num_suppliers):
        solution_dict[supplier] = Order[supplier].value()

    return solution_dict

# Single Group of Correlated Suppliers

Let's first consider the case where the buyer has reason to believe that all of the available suppliers are subject to correlated disruptions. Such a case may arise if all suppliers are located in the same country, all suppliers utilize a common raw material, or all suppliers utilize common shipment modes. The following code block reads in a dataset for such a case and shows how the correlation imputed by the dataset changes as the amount of data available varies.

In [ ]:
single_group_sample_filepath = pathlib.Path('data', 'single_group.csv')
single_group_sample = pd.read_csv(single_group_sample_filepath)
cutoff_list = [10, 15, 20, 25, 50, 100, 250, 500, 1000, 5000, 10000]

@interact(cutoff = cutoff_list)

def interactive_spend_analysis(cutoff = cutoff_list[0]):

    fig, ax = plt.subplots(1, 2, sharey = True, figsize = (16,6))

    sns.heatmap(single_group_sample[:cutoff].corr().values, 
                ax = ax[0], 
                vmin=-1, 
                vmax=1, 
                linewidths=0.1,
                linecolor='white',
                cmap = 'coolwarm')
    sns.heatmap(single_group_sample.corr().values, 
                ax = ax[1], 
                vmin=-1, 
                vmax=1,
                linewidths=0.1,
                linecolor='white',
                cmap = 'coolwarm')
    ax[0].set_title('Imputed Correlation', fontsize = 14)
    ax[1].set_title('True Correlation', fontsize = 14)
    
    plt.show()

The following code block calculates the on-time delivery rates for each supplier, specifies a unit cost vector for the available suppliers, and creates a list of the supplier names.

In [ ]:
marginals = single_group_sample.mean(axis=0).values
cost_vector = np.array([0.74, 0.83, 0.80, 0.87, 0.76])
supplier_names = list(single_group_sample.columns)

The following code block plots the unit cost and on-time delivery rate for each supplier.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8,6))
ax.scatter(x = marginals, 
           y = cost_vector,
           alpha = 0.7,
           edgecolor = 'k',
           s = 100,
          )
ax.set_xlabel('OTD')
ax.set_ylabel('Unit Cost')
ax.set_xlim(min(marginals)-0.05, max(marginals)+0.05)
ax.set_ylim(min(cost_vector)-0.05, max(cost_vector)+0.05)

for (otd_val, cost_val, supplier) in zip(marginals, cost_vector, supplier_names):
    ax.annotate(supplier, (otd_val + 0.005, cost_val + 0.005), fontsize = 12)

plt.show()

The following code block demonstrates how to determine the worst-case supply distribution for each supplier and uses it to solve the stochastic optimization problem. The dropdown box allows users to view how the solution changes as the amount of data used to fit the worst-case distribution varies.

In [ ]:
cutoff_list = [5, 10, 15, 20, 25, 50, 100, 1000, 5000, 10000]

@interact(cutoff = cutoff_list)

def interactive_spend_analysis(cutoff = cutoff_list[0]):
    
    np.random.seed(0)
    sample_size = 1000
    demand_sample = np.random.normal(loc = 1000, scale = 100, size = sample_size)

    single_group_data = single_group_sample[:cutoff].copy()
    estimated_marginals = single_group_data.mean(axis=0).values
        
    group_sample = np.random.rand(sample_size)

    otd_sample = np.ones((sample_size, len(estimated_marginals)))

    for column in range(otd_sample.shape[1]):
        otd_sample[:, column] = (group_sample <= estimated_marginals[column]).astype(int)

    solution = solve_OTD_model(cost_vector, otd_sample, demand_sample, 1.0)

    suppliers = list(solution.keys())
    orders = list(solution.values())

    fig, ax = plt.subplots(1, 1, figsize = (8,6))
    ax.scatter(x = marginals, 
               y = cost_vector, 
               alpha = 0.7,
               edgecolor = 'k',
               s = orders)
    ax.set_xlabel('OTD')
    ax.set_ylabel('Unit Cost')
    ax.set_xlim(min(marginals)-0.05, max(marginals)+0.05)
    ax.set_ylim(min(cost_vector)-0.05, max(cost_vector)+0.05)

    for (otd_val, cost_val, supplier) in zip(marginals, cost_vector, supplier_names):
        ax.annotate(supplier, (otd_val + 0.005, cost_val + 0.005), fontsize = 12)

    plt.show()

# Two Groups of Correlated Suppliers

We will now consider the case where the buying firm believes that the available suppliers can be partitioned into groups, where the suppliers in each group are subject to correlated disruptions. The following code block reads in a dataset for such a case and shows how the correlation imputed by the dataset changes as the amount of data available varies.

In [ ]:
two_group_sample_filepath = pathlib.Path('data', 'two_groups.csv')
two_group_sample = pd.read_csv(two_group_sample_filepath)
cutoff_list = [10, 15, 20, 25, 50, 100, 250, 500, 1000, 5000, 10000]

@interact(cutoff = cutoff_list)

def interactive_spend_analysis(cutoff = cutoff_list[0]):

    fig, ax = plt.subplots(1, 2, sharey = True, figsize = (16,6))

    sns.heatmap(two_group_sample[:cutoff].corr().values, 
                ax = ax[0], 
                vmin = -1, 
                vmax = 1, 
                linewidths = 0.1,
                linecolor = 'white',
                cmap = 'coolwarm')
    sns.heatmap(two_group_sample.corr().values, 
                ax = ax[1], 
                vmin = -1, 
                vmax = 1,
                linewidths = 0.1,
                linecolor = 'white',
                cmap = 'coolwarm')
    ax[0].set_title('Imputed Correlation', fontsize = 14)
    ax[1].set_title('True Correlation', fontsize = 14)
    
    plt.show()

The following code block calculates the on-time delivery rates for each supplier, specifies a unit cost vector for the available suppliers, and creates a list of the supplier names.

In [ ]:
marginals = two_group_sample.mean(axis=0).values
cost_vector = np.array([0.73, 0.85, 0.77, 0.75, 0.725, 0.71])
supplier_names = list(two_group_sample.columns)

The following code block plots the unit cost and on-time delivery rate for each supplier.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8,6))
ax.scatter(x = marginals, 
           y = cost_vector,
           alpha = 0.7,
           edgecolor = 'k',
           s = 100,
          )
ax.set_xlabel('OTD')
ax.set_ylabel('Unit Cost')
ax.set_xlim(min(marginals)-0.05, max(marginals)+0.05)
ax.set_ylim(min(cost_vector)-0.05, max(cost_vector)+0.05)

for (otd_val, cost_val, supplier) in zip(marginals, cost_vector, supplier_names):
    ax.annotate(supplier, (otd_val + 0.005, cost_val + 0.005), fontsize = 12)

plt.show()

The following code block demonstrates how to determine the worst-case supply distribution for the case with supplier groups and uses it to solve the stochastic optimization problem. The dropdown box allows users to view how the solution changes as the amount of data used to fit the worst-case distribution varies.

In [ ]:
cutoff_list = [5, 10, 15, 20, 25, 50, 100, 1000, 10000]

@interact(cutoff = cutoff_list)

def interactive_spend_analysis(cutoff = cutoff_list[0]):

    np.random.seed(0)
    sample_size = 1000
    demand_sample = np.random.normal(loc = 1000, scale = 100, size = sample_size)
    
    two_group_data = two_group_sample[:cutoff].copy()
    estimated_marginals = two_group_data.mean(axis=0).values

    np.random.seed(0)
    groups = [0,0,0,1,1,1]
    group_rvs = np.random.rand(sample_size, len(set(groups)))

    correlated_otd_sample = np.ones((sample_size, len(estimated_marginals)))
    
    for column in range(correlated_otd_sample.shape[1]):
        correlated_otd_sample[:,column] = (group_rvs[:,groups[column]] <= estimated_marginals[column]).astype(int)

    solution = solve_OTD_model(cost_vector, correlated_otd_sample, demand_sample, 1.0)

    suppliers = list(solution.keys())
    orders = list(solution.values())

    fig, ax = plt.subplots(1, 1, figsize = (8,6))
    ax.scatter(x = marginals, 
               y = cost_vector, 
               alpha = 0.7,
               edgecolor = 'k',
               s = orders)
    ax.set_xlabel('OTD')
    ax.set_ylabel('Unit Cost')
    ax.set_xlim(min(marginals)-0.05, max(marginals)+0.05)
    ax.set_ylim(min(cost_vector)-0.05, max(cost_vector)+0.05)

    for (otd_val, cost_val, supplier) in zip(marginals, cost_vector, supplier_names):
        ax.annotate(supplier, (otd_val + 0.005, cost_val + 0.005), fontsize = 12)

    plt.show()